In [ ]:
# 🔹 Upload dos arquivos necessários no Colab
from google.colab import files

print("👉 Escolha o arquivo dados.txt")
uploaded = files.upload()

print("👉 Escolha o arquivo monitoramento.xlsx")
uploaded = files.upload()

print("✅ Arquivos enviados com sucesso!")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from openpyxl import load_workbook

# Exemplo de leitura do TXT
with open("dados.txt", "r", encoding="utf-8") as f:
    linhas = f.readlines()[1:]  # ignora cabeçalho BASE-SVN

print("Linhas lidas do TXT:", linhas[:5])

In [ ]:
# Exemplo de abertura do Excel
wb = load_workbook("monitoramento.xlsx")
print("Abas disponíveis:", wb.sheetnames)

In [ ]:
# Aqui você continua com o código do seu notebook original
# (análises, gráficos, etc.)